# Import dependencies

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_csv('./restaurants.csv')
df.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


# Analysing and Cleaning Data

In [2]:
df.isnull().sum()

id                  0
position            0
name                0
score           24382
ratings         24382
category           54
price_range      9504
full_address      396
zip_code          429
lat                 0
lng                 0
dtype: int64

In [3]:
# Dropping unnecesary columns
df_clean=df.drop(columns=['zip_code','id','position'])

# Dropping rows with null values in essential columns 
df_clean = df_clean.dropna(subset=['category', 'full_address','price_range'])
df_clean = df_clean.reset_index()
df_clean = df_clean.drop(columns=['index'])

In [4]:
df_clean.head()

,name,score,ratings,category,price_range,full_address,lat,lng
0,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",33.562365,-86.830703
1,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",33.509800,-86.854640
2,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",33.404439,-86.806614
3,Ocean Restaurant,NaN,NaN,"Seafood, Sushi, Steak",$$,"1218 20th Street South, Birmingham, AL, 35205",33.498630,-86.795740
4,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",33.480440,-86.790440


# Data Engineering - Playing with features!

## category

In [5]:
# Generating dictionary of different categories of restaurants and their count

categories={}

for i in tqdm(range(26541)):
    temp=df_clean['category'][i].split(',')
    for j in temp:
        j=j.lower().strip()
        j=j.replace(" ","-")
        if j not in categories:
            categories[j]=1
        else:
            categories[j]+=1

categories = dict(sorted(categories.items(), key=lambda item: item[1],reverse=True))

100%|██████████| 26541/26541 [00:00<00:00, 115815.84it/s]


In [6]:
print(categories)

{'american': 12576, 'burgers': 4905, 'fast-food': 4595, 'sandwich': 3608, 'healthy': 3221, 'pizza': 3109, 'sandwiches': 3052, 'family-meals': 2969, 'comfort-food': 2603, 'breakfast-and-brunch': 2512, 'asian': 2441, 'desserts': 2388, 'mexican': 2225, 'convenience': 2184, 'everyday-essentials': 2145, 'salads': 2041, 'wings': 1993, 'italian': 1736, 'chicken': 1713, 'snacks': 1634, 'family-friendly': 1523, 'home-&amp;-personal-care': 1202, 'chinese': 1072, 'seafood': 1017, 'latin-american': 990, 'asian-fusion': 943, 'vegetarian': 920, 'japanese': 860, 'burger': 853, 'coffee-and-tea': 839, 'bakery': 820, 'pasta': 805, 'new-mexican': 798, 'vegetarian-friendly': 788, 'exclusive-to-eats': 703, 'cafe': 684, 'noodles': 681, 'grocery': 681, 'group-friendly': 670, 'juice-and-smoothies': 623, 'bbq': 618, 'pharmacy': 606, 'ice-cream-+-frozen-yogurt': 602, 'fried-chicken': 595, 'alcohol': 588, 'sushi': 582, 'kids-friendly': 553, 'burritos': 548, 'steak': 544, 'indian': 527, 'mediterranean': 505, 'tha

In [7]:
# Storing first 3 categories in new columns
def getCat1(row):
    temp = row['category'].split(',')
    return temp[0].lower().strip().replace(" ","-")

def getCat2(row):
    temp = row['category'].split(',')
    if len(temp)>1:
        return temp[1].lower().strip().replace(" ","-")
    else:
        return ''

def getCat3(row):
    temp = row['category'].split(',')
    if len(temp)>2:
        return temp[2].lower().strip().replace(" ","-")
    else:
        return ''

df_clean['primary_cat'] = df_clean.apply(lambda row: getCat1(row), axis=1)
df_clean['secondary_cat'] = df_clean.apply(lambda row: getCat2(row), axis=1)
df_clean['tertiary_cat'] = df_clean.apply(lambda row: getCat3(row), axis=1)

In [8]:
df_clean.head()

,name,score,ratings,category,price_range,full_address,lat,lng,primary_cat,secondary_cat,tertiary_cat
0,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",33.562365,-86.830703,burgers,american,sandwiches
1,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",33.509800,-86.854640,american,cheesesteak,sandwiches
2,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",33.404439,-86.806614,pizza,,
3,Ocean Restaurant,NaN,NaN,"Seafood, Sushi, Steak",$$,"1218 20th Street South, Birmingham, AL, 35205",33.498630,-86.795740,seafood,sushi,steak
4,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",33.480440,-86.790440,sushi,asian,japanese


## price_range

In [9]:
df_clean['price_range'].unique()

array(['$', '$$', '$$$', '$$$$', '$$$$$$$$$$$$$$$$$'], dtype=object)

In [10]:
# Encoding the categorical values numerically
label_encoder = LabelEncoder()

df_clean['price_encoded'] = label_encoder.fit_transform(df_clean['price_range'])

In [11]:
df_clean.head()

,name,score,ratings,category,price_range,full_address,lat,lng,primary_cat,secondary_cat,tertiary_cat,price_encoded
0,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",33.562365,-86.830703,burgers,american,sandwiches,0
1,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",33.509800,-86.854640,american,cheesesteak,sandwiches,0
2,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",33.404439,-86.806614,pizza,,,0
3,Ocean Restaurant,NaN,NaN,"Seafood, Sushi, Steak",$$,"1218 20th Street South, Birmingham, AL, 35205",33.498630,-86.795740,seafood,sushi,steak,1
4,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",33.480440,-86.790440,sushi,asian,japanese,0


## full_address

In [12]:
# Extracting city and state name from address
def getAddress(row):  
    return ','.join(row['full_address'].split(',')[-3:-1]).strip()

df_clean['address'] = df_clean.apply(lambda row: getAddress(row), axis=1)

In [13]:
df_clean.head()

,name,score,ratings,category,price_range,full_address,lat,lng,primary_cat,secondary_cat,tertiary_cat,price_encoded,address
0,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",33.562365,-86.830703,burgers,american,sandwiches,0,"Birmingham, AL"
1,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",33.509800,-86.854640,american,cheesesteak,sandwiches,0,"Birmingham, AL"
2,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",33.404439,-86.806614,pizza,,,0,"Hoover, AL"
3,Ocean Restaurant,NaN,NaN,"Seafood, Sushi, Steak",$$,"1218 20th Street South, Birmingham, AL, 35205",33.498630,-86.795740,seafood,sushi,steak,1,"Birmingham, AL"
4,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",33.480440,-86.790440,sushi,asian,japanese,0,"Birmingham, AL"


# Word embeddings

In [14]:
# Function to generate word embeddings for a text column
def generate_word_embeddings(column):
    unique_values = df_clean[column].unique()
    sentences = [[value] for value in unique_values]
    model = Word2Vec(sentences, min_count=1, vector_size=100)
    return model

# Define the text columns for which word embeddings need to be generated
text_columns = ['primary_cat','secondary_cat','tertiary_cat']

# Dictionary to store the models
word_embedding_models = {}

# Generate and store word embeddings for each text column
for column in text_columns:
    model = generate_word_embeddings(column)
    word_embedding_models[column] = model

# Apply word embeddings to correct columns and create new columns for embeddings
for column, model in tqdm(word_embedding_models.items(), desc="embeddings"):
    df_clean[column + '_embeddings'] = df_clean[column].apply(lambda x: model.wv[x].reshape(1, -1) if x in model.wv else [])

embeddings: 100%|██████████| 3/3 [00:00<00:00, 10.65it/s]


In [15]:
df_clean.head()

,name,score,ratings,category,price_range,full_address,lat,lng,primary_cat,secondary_cat,tertiary_cat,price_encoded,address,primary_cat_embeddings,secondary_cat_embeddings,tertiary_cat_embeddings
0,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",33.562365,-86.830703,burgers,american,sandwiches,0,"Birmingham, AL","[[0.009382484, -0.004806672, 0.009592734, 0.00...","[[-0.009488757, 0.005276706, -0.0002366674, 0....","[[-0.002866646, -0.0058289613, 0.006462299, 0...."
1,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",33.509800,-86.854640,american,cheesesteak,sandwiches,0,"Birmingham, AL","[[0.0042227684, 0.00017856836, 0.0047846828, 0...","[[-0.004473716, -0.00014054537, 0.0044175754, ...","[[-0.002866646, -0.0058289613, 0.006462299, 0...."
2,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",33.404439,-86.806614,pizza,,,0,"Hoover, AL","[[-0.009006461, 0.005336474, 0.0037595546, -0....","[[-0.004021951, 0.0025012803, 0.0015762186, -0...","[[-0.009902985, -0.00042589306, -0.0047100745,..."
3,Ocean Restaurant,NaN,NaN,"Seafood, Sushi, Steak",$$,"1218 20th Street South, Birmingham, AL, 35205",33.498630,-86.795740,seafood,sushi,steak,1,"Birmingham, AL","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.007291925, -0.00071569445, 0.005721662, -0...","[[0.0059583248, 0.0037787259, 0.0017718363, 0...."
4,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",33.480440,-86.790440,sushi,asian,japanese,0,"Birmingham, AL","[[-0.009902985, -0.00042589306, -0.0047100745,...","[[0.009379394, 0.0073653986, 0.006877606, 0.00...","[[-0.0022360254, 0.00751379, -0.003086332, 0.0..."


# Feature Weightage

In [16]:
weightage = {
   'primary_cat_embeddings':1.0,
   'secondary_cat_embeddings':0.8,
   'tertiary_cat_embeddings':0.5,
   'price_encoded':0.5
}

You must be wondering, why haven't we used score (rating) of the restaurants. We can see from the Power BI dashboard and null values analysis that.
1. Many restaurants do not have a score
2. The average scoring for most of the restaurants is extremely high and this might lead skewing of results

# Generating recommendations

In [17]:
# Calculate similarity scores for 'price' column using cosine similarity
def calculate_price_similarity(price1, price2):
    price1 = np.array(price1).reshape(1, -1)
    price2 = np.array(price2).reshape(1, -1)
    similarity = cosine_similarity(price1, price2)
    return similarity[0][0]

def calculate_text_similarity(value1, value2):
    similarity = cosine_similarity(value1, value2)
    return similarity[0][0]

# Calculate weighted similarity between two restaurants
def calculate_weighted_similarity(restaurant1, restaurant2):
    similarity_scores = []
    for column, weight in weightage.items():
        if column == 'price_encoded':
            similarity = calculate_price_similarity(restaurant1[column].values[0], restaurant2[column])
        else:
            similarity = calculate_text_similarity(restaurant1[column].values[0], restaurant2[column])
        similarity_scores.append(similarity * weight)
    weighted_similarity = sum(similarity_scores)
    return weighted_similarity

In [18]:
search_for_recommendations = "Baskin-Robbins (4905 Cordell Ave)"

In [19]:
restaurant = df_clean[df_clean['name'] == search_for_recommendations].iloc[:1]

if restaurant['secondary_cat'].values[0] == '':
    del weightage['secondary_cat_embeddings']
if restaurant['tertiary_cat'].values[0] == '':
    del weightage['tertiary_cat_embeddings']

new_df_clean = df_clean.copy(deep=True)
new_df_clean['similarity_score'] = df_clean.apply(lambda x: calculate_weighted_similarity(restaurant, x), axis=1)
top_10_similar_restaurants = new_df_clean.nlargest(10, 'similarity_score')

In [20]:
top_10_similar_restaurants

,name,score,ratings,category,price_range,full_address,lat,lng,primary_cat,secondary_cat,tertiary_cat,price_encoded,address,primary_cat_embeddings,secondary_cat_embeddings,tertiary_cat_embeddings,similarity_score
436,Baskin-Robbins (3064 Ross Clark Cir),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"3064 Ross Clark Cir, Dothan, AL, 36301",31.225890,-85.431810,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Dothan, AL","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
687,Baskin-Robbins (1024 6th Ave Se),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"1024 6th Ave SE, Decatur, AL, 35601",34.591420,-86.979940,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Decatur, AL","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
816,"Baskin-Robbins (5000 Whitesburg Dr S, Suite 156)",5.0,42.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"5000 Whitesburg Dr SW, Huntsville, AL, 35802",34.690520,-86.569150,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Huntsville, AL","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
982,Baskin-Robbins (723 E 16th St),4.2,13.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"723 E 16th St, Cheyenne, WY, 82001",41.135765,-104.806351,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Cheyenne, WY","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
1390,Baskin-Robbins (1221 Lombardi Access Rd),4.9,15.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"1221 Lombardi Access Rd, Green Bay, WI, 54304",44.505790,-88.072350,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Green Bay, WI","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
1979,Baskin-Robbins (8820 N Port Washington Rd),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"8820 N Port Washington Rd, Bayside, WI, 53217",43.177575,-87.912511,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Bayside, WI","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
2009,Baskin-Robbins (1417 N. Wauwatosa Ave),4.8,56.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"1417 N., Wauwatosa, WI, 53213",43.050200,-88.007666,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Wauwatosa, WI","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
2366,Baskin-Robbins (1610 W Washington St),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"1610 W Washington St, West Bend, WI, 53095",43.427470,-88.198270,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"West Bend, WI","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
2452,Baskin-Robbins (110 W Bridge St),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"110 W Bridge St, Wausau, WI, 54401",44.970230,-89.637760,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Wausau, WI","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3
2927,Baskin-Robbins (6026 Washington Ave),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"6026 Washington Ave, Mt Pleasant, WI, 53406",42.719650,-87.853890,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Mt Pleasant, WI","[[-0.0022360

Voila! We have recommended restaurants similar to that of our choice. But there still something missing. 

For a realistic UberEats recommendation system, we need to establish a function to generate restaurants within a some 'n' kilometer radius.

# Restaurants within radius

In [21]:
def get_areas_within_radius(df, latitude, longitude, radius_km=10):
    """
    Get row details of areas within a specified radius from a given latitude and longitude.

    Parameters:
    - df: DataFrame containing area details.
    - latitude: Latitude of the target location.
    - longitude: Longitude of the target location.
    - radius_km: Radius in kilometers (default is 10 km).

    Returns:
    - DataFrame containing areas within the specified radius.
    """
    # Radius of the Earth in kilometers
    earth_radius_km = 6371.0

    # Calculate the differences in latitude and longitude
    df['lat_diff'] = np.radians(df['lat'] - latitude)
    df['lng_diff'] = np.radians(df['lng'] - longitude)

    # Calculate Haversine distance
    a = np.sin(df['lat_diff'] / 2)**2 + np.cos(np.radians(latitude)) * np.cos(np.radians(df['lat'])) * np.sin(df['lng_diff'] / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distances_km = earth_radius_km * c

    # Filter areas within the specified radius
    result_df = df[distances_km <= radius_km].drop(columns=['lat_diff', 'lng_diff'])

    result_df['distance_km'] = distances_km[distances_km <= radius_km]

    return result_df


latitude = restaurant['lat'].values[0]
longitude = restaurant['lng'].values[0]
areas_within_radius = get_areas_within_radius(new_df_clean, latitude, longitude, radius_km=25)

In [22]:
# Sorting by Similarity and Closest distance
k=areas_within_radius.nlargest(11, ['similarity_score'])
top10_similar_nearby_restaurants = k.nsmallest(11, ['distance_km'])

In [23]:
# Similar restaurants in a 10km radius
top10_similar_nearby_restaurants[1:]

,name,score,ratings,category,price_range,full_address,lat,lng,primary_cat,secondary_cat,tertiary_cat,price_encoded,address,primary_cat_embeddings,secondary_cat_embeddings,tertiary_cat_embeddings,similarity_score,distance_km
18463,Baskin-Robbins (8119 Watson St),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"8119 Watson St, McLean, VA, 22102",38.917170,-77.227320,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"McLean, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,13.734696
19663,Baskin-Robbins (3100 Columbia Pike),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"3100 Columbia Pike, Arlington, VA, 22204",38.861840,-77.088700,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Arlington, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,14.198782
18985,Baskin-Robbins (6664 Arlington Blvd),4.6,14.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"6664 Arlington Blvd, Falls Church, VA, 22042",38.870246,-77.174864,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Falls Church, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,14.799951
16883,Baskin-Robbins (5701 Columbia Pike),4.1,23.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"5701 Columbia Pike, Falls Church, VA, 22041",38.851090,-77.125960,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Falls Church, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,15.553334
17200,Baskin-Robbins (3325 Jefferson Davis Hwy),4.4,19.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"3325 Jefferson Davis Hwy, Alexandria, VA, 22305",38.835310,-77.051280,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Alexandria, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,17.602445
16742,Baskin-Robbins (1701 Centre Plz),5.0,14.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"1701 Centre Plz, Alexandria, VA, 22302",38.829174,-77.085015,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Alexandria, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,17.843878
18592,"Baskin-Robbins (1446 Reston Pkwy, North Point ...",4.7,35.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"1446 Reston Pkwy, North Point Village Center, ...",38.979949,-77.349187,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Hunter Mill, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,21.702237
21118,Baskin-Robbins (18006 Mateny Rd),4.4,15.0,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"18006 Mateny Rd, Germantown, MD, 20874",39.146500,-77.274280,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Germantown, MD","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,23.153534
18877,Baskin-Robbins (10657 Fairfax Blvd),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"10657, Fairfax, VA, 22030",38.856274,-77.312276,desserts,ice-cream-+-frozen-yogurt,comfort-food,0,"Fairfax, VA","[[-0.0022360254, 0.00751379, -0.003086332, 0.0...","[[0.005028101, -0.003685106, 0.0034227418, 0.0...","[[0.00307577, -0.0015842546, -0.001196326, -0....",2.3,23.689541
19341,Baskin-Robbins (47010 Community Plz Suite 180),NaN,NaN,"Desserts, Ice Cream + Frozen Yogurt, Comfort Food",$,"47

Yay! Successfully built a simple recommender system for restaurants.